<a href="https://colab.research.google.com/github/jusiahk/StandardBankLoanDefault/blob/master/seniorDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem: How can we grade clients by there credit scores to determine there credit risk/worthiness.

Solution: Creadit Score Card to manage, determine a borrower who is able to pay back there loan before 60 days elapse. (Credit Score) is an assessment of the relative likelihood of a certain credit event occurring, given a number of observable inputs.

Can be used to decide whether a client's credit application is accepted or rejected basing on there 3 credit score figure.


In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score




Explanatory Data Analysis  & Processing

In [3]:
# Load the dataset
data = pd.read_excel('C:/Users/hp/Desktop/noahk - Copy.xlsx')

FileNotFoundError: ignored

In [ ]:
df = data
df.shape # shape of the complete dataset

In [ ]:
#Let's start by examining the structure of the dataset
print("Dataset shape:", data.shape)
print("Columns:", data.columns)

In [ ]:
df.isna().sum() # checking for null values

In [ ]:
#lets check and treat any duplicates.
df.duplicated()

In [ ]:
#Checking for dupplicates
# Identify duplicate rows across all columns
duplicates = df[df.duplicated(keep='first')]

# Drop the duplicate rows, keeping the first occurrence
df.drop_duplicates(keep='first', inplace=True)

# Append one occurrence of each duplicate back to the DataFrame
df = df.append(duplicates.head(1))

# Reset the index of the modified DataFrame
df.reset_index(drop=True, inplace=True)

In [ ]:
# Re-explore the df
df.info()

In [ ]:
df.skew()

In [ ]:
df.var()

In [ ]:
# Assuming any loan_age_day above 60 is defaulted =1 otherwise false 0 a variable is created as target variable
data['Response'] = np.where(data['LOAN_AGE_DAYS'] > 60, 1, 0)
# Drop the original 'loan_status' column
print(data)

In [ ]:
#lets see the head of our dataset the 5 columns
df.head()

In [ ]:
#Summary statistics of numerical columns
df = data
print("\nSummary statistics:")
df.describe().transpose()

In [ ]:
#let filter out for the bill values greater than 40 dollars
df[df["RCVD_AMNT_ ON_WALLET"] == 0.0]

In [ ]:
#let filter out for the response values greater than 60
df[df["Response"] == 1]

Dealing with outliers far from the mean

In [ ]:
# lets check our dataset for  outliers and normalize if any
# Calculate z-scores for each variable
z_scores = np.abs(stats.zscore(df))

# Set a threshold for outlier detection
threshold = 3

# Identify the outliers by comparing z-scores to the threshold
outlier_indices = np.where(z_scores > threshold)

# Create a DataFrame with the outlier values
outliers = pd.DataFrame({'Variable': df.columns[outlier_indices[1]],
                         'Outlier Value': df.values[outlier_indices]})

# Display the outlier table
print(outliers)

In [ ]:
#Can we normalize these as we have data withmany zeros but also helps on sparsity.
df['LOAN_AGE_DAYS'].value_counts(normalize = True)
df['WALLET_CREDITS'].value_counts(normalize = True)
df['TOTAL AIRTIME LOAN'].value_counts(normalize = True)
df['TRSF_FROM_BANK_TO_ WALLET _AMNT'].value_counts(normalize = True)

Feature selection & Target variable Identification

In [ ]:
# define features and labels
features = list(df.columns.values)
features.remove('Response')
labels = ['Response']

In [ ]:
X = df[features]
y = df[labels]

In [ ]:
cor = df.corr()
plt.figure(figsize = (10,6))
sns.heatmap(cor, annot = True)

Model Building & Evaluation

In [ ]:
#Machine learning models expect scaled data in range of 1 or 0 .
scaler = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2,random_state=9)

pd.DataFrame(scaled_X, columns=X.columns.values).head()

a) The statement might be true basing on a a specific dataset. Supprisingly with ths dataset biggest percentage falls
between 14 and 17th of the month.

Though in normal cases one is advised not to imply due to any biases or miss-reprentation of data could affect performance
hence its advised to look at a whole dataset.


In [ ]:
plt.subplot(1,2,1)
plt.bar(Counter(y_train['Response']).keys(),Counter(y_train['Response']).values())
plt.xticks([0,1])
plt.title('Training set')
plt.subplot(1,2,2)
plt.bar(Counter(y_test['Response']).keys(),Counter(y_test['Response']).values())
plt.xticks([0,1])
plt.title('Test set')
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model and weights will put into account the imbalances
clf = LogisticRegression(random_state=0, max_iter=1000, class_weight='balanced')

# Perform cross-validation for better nodel performance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores (Accuracy):", scores)
print("Mean Accuracy:", scores.mean())

# Fit the logistic regression model
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc * 100)

# Print confusion matrix
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(confusion_matrix=result).plot()
plt.show()

# Print classification report
result = classification_report(y_test, y_pred)
print("Classification Report:")
print(result)

Score Card creation

In [ ]:
# Convert the numpy array back to a DataFrame
X_test_df = pd.DataFrame(X_test, columns=['CUST_TXN_DATE', 'CUST_ID_ACCT1', 'DEPOSIT_AMNT_ON_WALLET', 'RCVD_AMNT_ON_WALLET',
                                          'TRSF_FROM_BANK_TO_WALLET_AMNT', 'WALLET_CREDITS', 'DAYS_ARTM_LESS_UGX 2000',
                                          'TOTAL AIRTIME LOAN', 'LOAN_AGE_DAYS'])

# Extract the customer IDs from the test dataset
cust_ids = X_test_df['CUST_ID_ACCT1']

# Preprocess the test dataset by selecting relevant features and encoding categorical variables if needed
X_test_processed = X_test_df[['CUST_TXN_DATE', 'CUST_ID_ACCT1', 'DEPOSIT_AMNT_ON_WALLET', 'RCVD_AMNT_ON_WALLET',
                              'TRSF_FROM_BANK_TO_WALLET_AMNT', 'WALLET_CREDITS', 'DAYS_ARTM_LESS_UGX 2000',
                              'TOTAL AIRTIME LOAN', 'LOAN_AGE_DAYS']]

# Calculate credit scores using the trained model
credit_scores = clf.predict_proba(X_test_processed)[:, 1] * 1000

# Define credit score grade thresholds
grade_thresholds = [300, 550, 800, 850]

# Assign credit score grades based on thresholds
credit_score_grades = pd.cut(credit_scores, bins=grade_thresholds, labels=['D', 'C', 'B'], right=True)


# Create a dataframe with customer ID and credit score (without credit_score_grades)
credit_score_df = pd.DataFrame({'CUST_ID_ACCT1': cust_ids, 'credit_score': credit_scores})

# Print the credit score information
print(credit_score_df)

In [ ]:
# Define the minimum and maximum scores you want
min_score = 300
max_score = 850

# Calculate credit scores using the trained model
credit_scores = clf.predict_proba(X_test_processed)[:, 1]

# Rescale the credit scores to the desired range
scaled_credit_scores = min_score + (max_score - min_score) * (credit_scores - credit_scores.min()) / (credit_scores.max() - credit_scores.min())

# Convert the scaled credit scores to integers
credit_score_df['credit_score'] = scaled_credit_scores.astype(int)

# Print the credit score information
print(credit_score_df)
